In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

/var/folders/kq/ps63fdtn2yvcch21qn3fl_t80000gn/T/ipykernel_8597/1123596123.py:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


In [4]:
data = pd.read_csv("bitcoin.csv", parse_dates = ["Date"], index_col = "Date")
data

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807
...,...,...,...,...,...
2021-10-07 05:00:00,55073.20,55073.21,54545.07,54735.76,2251.122020
2021-10-07 06:00:00,54735.77,54968.06,54375.83,54534.16,1783.004260
2021-10-07 07:00:00,54534.16,54793.26,54235.33,54755.92,4163.431360


In [7]:
data.info()
data["returns"] = np.log(data.Close/data.Close.shift(1))
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 36168 entries, 2017-08-17 04:00:00 to 2021-10-07 09:00:00
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Open     36168 non-null  float64
 1   High     36168 non-null  float64
 2   Low      36168 non-null  float64
 3   Close    36168 non-null  float64
 4   Volume   36168 non-null  float64
 5   returns  36167 non-null  float64
dtypes: float64(6)
memory usage: 2.9 MB


,Open,High,Low,Close,Volume,returns
Date,,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,NaN
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,0.001505
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,0.002090
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,0.005912
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,0.002457
...,...,...,...,...,...,...
2021-10-07 05:00:00,55073.20,55073.21,54545.07,54735.76,2251.122020,-0.006146
2021-10-07 06:00:00,54735.77,54968.06,54375.83,54534.16,1783.004260,-0.003690
2021-10-07 07:00:00,54534.16,54793.26,54235.33,54755.92,4163.431360,0.004058


In [14]:
from pyrsistent import thaw


def backtest(data, parameter, tc):
    # prepare features
    data = data[["Close", "Volume", "returns"]].copy()
    data["vol_ch"] = np.log(data.Volume.div(data.Volume.shift(1)))
    data.loc[data.vol_ch > 3, "vol_ch"] = np.nan
    data.loc[data.vol_ch < -3 , "vol_ch"] = np.nan

    # define trading postions
    return_th = np.percentile(data.returns.dropna(), parameter[0])
    cond1 = data.returns >= return_th
    volume_th = np.percentile(data.vol_ch.dropna(), [parameter[1], parameter[2]])
    cond2 = data.vol_ch.between(volume_th[0], volume_th[1])

    data["position"] = 1
    data.loc[cond1 & cond2, "position"] = 0

    # backtest
    data["strategy"] = data.position.shift(1)* data["returns"]
    data["trades"] = data.position.diff().fillna(0).abs()
    data.strategy = data.strategy + data.trades * tc
    data["creturns"] = data["returns"].cumsum().apply(np.exp)
    data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)

    return data.cstrategy[-1]


In [15]:
import warnings
warnings.filterwarnings("ignore")

In [22]:
backtest(data = data, parameter = (90, 5, 20), tc = -0.01)

0.02672089501616669

In [23]:
return_range = range(85, 98, 1) # potential values for return_th
vol_low_range = range(2, 16, 1) # potential values for vol_low
vol_high_range = range(16, 35,1) # potential vaues for vol_high

In [24]:
list(return_range)

[85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97]

In [25]:
# run backtest on all ranges and find the combination with the best result

from itertools import product

In [26]:
combinations = list(product(return_range, vol_low_range, vol_high_range))
combinations

[(85, 2, 16),
 (85, 2, 17),
 (85, 2, 18),
 (85, 2, 19),
 (85, 2, 20),
 (85, 2, 21),
 (85, 2, 22),
 (85, 2, 23),
 (85, 2, 24),
 (85, 2, 25),
 (85, 2, 26),
 (85, 2, 27),
 (85, 2, 28),
 (85, 2, 29),
 (85, 2, 30),
 (85, 2, 31),
 (85, 2, 32),
 (85, 2, 33),
 (85, 2, 34),
 (85, 3, 16),
 (85, 3, 17),
 (85, 3, 18),
 (85, 3, 19),
 (85, 3, 20),
 (85, 3, 21),
 (85, 3, 22),
 (85, 3, 23),
 (85, 3, 24),
 (85, 3, 25),
 (85, 3, 26),
 (85, 3, 27),
 (85, 3, 28),
 (85, 3, 29),
 (85, 3, 30),
 (85, 3, 31),
 (85, 3, 32),
 (85, 3, 33),
 (85, 3, 34),
 (85, 4, 16),
 (85, 4, 17),
 (85, 4, 18),
 (85, 4, 19),
 (85, 4, 20),
 (85, 4, 21),
 (85, 4, 22),
 (85, 4, 23),
 (85, 4, 24),
 (85, 4, 25),
 (85, 4, 26),
 (85, 4, 27),
 (85, 4, 28),
 (85, 4, 29),
 (85, 4, 30),
 (85, 4, 31),
 (85, 4, 32),
 (85, 4, 33),
 (85, 4, 34),
 (85, 5, 16),
 (85, 5, 17),
 (85, 5, 18),
 (85, 5, 19),
 (85, 5, 20),
 (85, 5, 21),
 (85, 5, 22),
 (85, 5, 23),
 (85, 5, 24),
 (85, 5, 25),
 (85, 5, 26),
 (85, 5, 27),
 (85, 5, 28),
 (85, 5, 29),
 (85, 

In [ ]:
results =  []
